# OCR Cookbook

---

## OCR Exploration and Structured Outputs
In this cookbook, we will explore the basics of OCR and leverage it together with existing models to achieve structured outputs fueled by our OCR model.

You may want to do this in case current vision models are not powerful enough, hence enhancing their vision OCR capabilities with the OCR model to achieve better structured data extraction.

---

### Used
- Mistral OCR
- Pixtral 12B & Ministral 8B


### Setup
First, let's install `mistralai` and download the required files.

In [1]:
!pip install mistralai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.3/278.3 kB 7.3 MB/s eta 0:00:00


In [ ]:
!wget https://raw.githubusercontent.com/mistralai/cookbook/refs/heads/main/mistral/ocr/mistral7b.pdf
!wget https://raw.githubusercontent.com/mistralai/cookbook/refs/heads/main/mistral/ocr/receipt.png

We can now set up our client. You can create an API key on our [Plateforme](https://console.mistral.ai/api-keys/).

In [2]:
from mistralai import Mistral

api_key = "API_KEY"
client = Mistral(api_key=api_key)

There are two types of files you can apply OCR to:
- PDF files, either uploaded or from URLs...

In [3]:
from pathlib import Path

pdf_file = Path("/content/30 images pdf.pdf")
assert pdf_file.is_file()

In [4]:
from mistralai import DocumentURLChunk, ImageURLChunk, TextChunk
import json

uploaded_file = client.files.upload(
    file={
        "file_name": pdf_file.stem,
        "content": pdf_file.read_bytes(),
    },
    purpose="ocr",
)

signed_url = client.files.get_signed_url(file_id=uploaded_file.id, expiry=1)

pdf_response = client.ocr.process(document=DocumentURLChunk(document_url=signed_url.url), model="mistral-ocr-latest", include_image_base64=True)

response_dict = json.loads(pdf_response.json())
json_string = json.dumps(response_dict, indent=4)
print(json_string)

{
    "pages": [
        {
            "index": 0,
            "markdown": "are political parties with differing aims, social sets, cliques, gangs, corporations, partnerships, groups bound closely together by ties of blood, and so on in endless variety. In many modern states and in some ancient, there is great diversity of populations, of varying languages, religions, moral codes, and traditions. From this standpoint, many a minor political unit, one of our large cities, for example, is a congeries of loosely associated societies, rather than an inclusive and permeating community of action and thought. (See ante, p. 20.)\n\nThe terms society, community, are thus ambiguous. They have both a eulogistic or normative sense, and a descriptive sense; a meaning de jure and a meaning de facto. In social philosophy, the former connotation is almost always uppermost. Society is conceived as one by its very nature. The qualities which accompany this unity, praiseworthy community of purpose and we

<ipython-input-4-793982fc17e2>:16: PydanticDeprecatedSince20: The `json` method is deprecated; use `model_dump_json` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  response_dict = json.loads(pdf_response.json())


*The OCR model can output interleaved text and images (set `include_image_base64=True` to return the base64 image ), we can view the result with the following:*

In [5]:
from mistralai.models import OCRResponse
from IPython.display import Markdown, display

def replace_images_in_markdown(markdown_str: str, images_dict: dict) -> str:
    for img_name, base64_str in images_dict.items():
        markdown_str = markdown_str.replace(f"![{img_name}]({img_name})", f"![{img_name}]({base64_str})")
    return markdown_str

def get_combined_markdown(ocr_response: OCRResponse) -> str:
  markdowns: list[str] = []
  for page in ocr_response.pages:
    image_data = {}
    for img in page.images:
      image_data[img.id] = img.image_base64
    markdowns.append(replace_images_in_markdown(page.markdown, image_data))

  return "\n\n".join(markdowns)

display(Markdown(get_combined_markdown(pdf_response)))

are political parties with differing aims, social sets, cliques, gangs, corporations, partnerships, groups bound closely together by ties of blood, and so on in endless variety. In many modern states and in some ancient, there is great diversity of populations, of varying languages, religions, moral codes, and traditions. From this standpoint, many a minor political unit, one of our large cities, for example, is a congeries of loosely associated societies, rather than an inclusive and permeating community of action and thought. (See ante, p. 20.)

The terms society, community, are thus ambiguous. They have both a eulogistic or normative sense, and a descriptive sense; a meaning de jure and a meaning de facto. In social philosophy, the former connotation is almost always uppermost. Society is conceived as one by its very nature. The qualities which accompany this unity, praiseworthy community of purpose and welfare, loyalty to public ends, mutuality of sympathy, are emphasized. But when we look at the facts which the term denotes instead of confining our attention to its intrinsic connotation, we find not unity, but a plurality of societies, good and bad. Men banded together in a criminal conspiracy, business aggregations that prey upon the public while serving it, political machines held together by the interest of plunder, are included. If

interests as a factor in social control. The second means not only freer interaction between social groups (once isolated so far as intention could keep up a separation) but change in social habitits continuous readjustment through meeting the new situations produced by varied intercourse. And these two traits are precisely what characterize the democratically constituted society.

Upon the educational side, we note first that the realization of a form of social life in which interests are mutually interpenetrating, and where progress, or readjustment, is an important consideration, makes a democratic community more interested than other communities have cause to be in deliberate and systematic education. The devotion of democracy to education is a familiar fact. The superficial explanation is that a government resting upon popular suffrage cannot be successful unless those who elect and who obey their governors are educated. Since a democratic society repudiates the principle of external authority, it must find a substitute in voluntary disposition and interest; these can be created only by education. But there is a deeper explanation. A democracy is more than a form of government; it is primarily a mode of associated living, of conjoint communicated experience. The

following nature was a political dogma. It meant a rebellion against existing social institutions, customs, and ideals (See ante, p. 91). Rousseau's statement that everything is good as it comes from the hands of the Creator has its signification only in its contrast with the concluding part of the same sentence: "Everything degenerates in the hands of man." And again he says: "Natural man has an absolute value; he is a numerical unit, a complete integer and has no relation save to himself and to his fellow man. Civilized man is only a relative unit, the numerator of a fraction whose value depends upon its dominator, its relation to the integral body of society. Good political institutions are those which make a man unnatural." It is upon this conception of the artificial and harmful character of organized social life as it now exists 2 that he rested the notion that nature not merely furnishes prime forces which initiate growth but also its plan and goal. That evil institutions and customs work almost automatically to give a wrong education which the most careful schooling cannot offset is true enough; but the conclusion is not to education apart from the environment, but to provide an environment in which native powers will be put to better uses.
2. Social Efficiency as Aim. A conception which made nature supply the end of a true education and society the end of an evil one, could hardly

intellectual opportunities are accessible to all on equable and easy terms. A society marked off into classes need he specially attentive only to the education of its ruling elements. A society which is mobile, which is full of channels for the distribution of a change occurring anywhere, must see to it that its members are educated to personal initiative and adaptability. Otherwise, they will be overwhelmed by the changes in which they are caught and whose significance or connections they do not perceive. The result will be a confusion in which a few will appropriate to themselves the results of the blind and externally directed activities of others.
3. The Platonic Educational Philosophy. Subsequent chapters will be devoted to making explicit the implications of the democratic ideas in education. In the remaining portions of this chapter, we shall consider the educational theories which have been evolved in three epochs when the social import of education was especially conspicuous. The first one to be considered is that of Plato. No one could better express than did he the fact that a society is stably organized when each individual is doing that for which he has aptitude by nature in such a way as to be useful to others (or to contribute to the whole to which he belongs); and that it is the business of education to discover these aptitudes and progressively to train them for social use.

assimilation of new presentations, their character is all important. The effect of new presentations is to reinforce groupings previously formed. The business of the educator is, first, to select the proper material in order to fix the nature of the original reactions, and, secondly, to arrange the sequence of subsequent presentations on the basis of the store of ideas secured by prior transactions. The control is from behind, from the past, instead of, as in the unfolding conception, in the ultimate goal.
(3) Certain formal steps of all method in teaching may be laid down. Presentation of new subject matter is obviously the central thing, but since knowing consists in the way in which this interacts with the contents already submerged below consciousness, the first thing is the step of "preparation,"-that is, calling into special activity and getting above the floor of consciousness those older presentations which are to assimilate the new one. Then after the presentation, follow the processes of interaction of new and old; then comes the application of the newly formed content to the performance of some task. Everything must go through this course; consequently there is a perfectly uniform method in instruction in all subjects for all pupils of all ages.

Herbart's great service lay in taking the work of teaching out of the region of routine and accident.

control. To say that one knows what he is about, or can intend certain consequences, is to say, of course, that he can better anticipate what is going to happen; that he can, therefore, get ready or prepare in advance so as to secure beneficial consequences and avert undesirable ones. A genuinely educative experience, then, one in which instruction is conveyed and ability increased, is contradistinguished from a routine activity on one hand, and a capricious activity on the other. (a) In the latter one "does not care what happens"; one just lets himself go and avoids connecting the consequences of one's act (the evidences of its connections with other things) with the act. It is customary to frown upon such aimless random activity, treating it as willful mischief or carelessness or lawlessness. But there is a tendency to seek the cause of such aimless activities in the youth's own disposition, isolated from everything else. But in fact such activity is explosive, and due to maladjustment with surroundings. Individuals act capriciously whenever they act under external dictation, or from being told, without having a purpose of their own or perceiving the bearing of the deed upon other acts. One may learn by doing something which he does not understand; even in the most intelligent action, we do much which we do not mean, because the largest portion of the connections of the act we consciously intend are not perceived or anticipated. But we learn only

Now for that of discipline. Where an activity takes time, where many means and obstacles lie between its initiation and completion, deliberation and persistence are required. It is obvious that a very large part of the everyday meaning of will is precisely the deliberate or conscious disposition to persist and endure in a planned course of action in spite of difficulties and contrary solicitations. A man of strong will, in the popular usage of the words, is a man who is neither fickle nor half-hearted in achieving chosen ends. His ability is executive; that is, he persistently and energetically strives to execute or carry out his aims. A weak will is unstable as water.

Clearly there are two factors in will. One has to do with the foresight of results, the other with the depth of hold the foreseen outcome has upon the person.
(I) Obstinacy is persistence but it is not strength of volition. Obstinacy may be mere animal inertia and insensitiveness. A man keeps on doing a thing just because he has got started, not because of any clearly thought-out purpose. In fact, the obstinate man generally declines (although he may not be quite aware of his refusal) to make clear to himself what his proposed end is; he has a feeling that if he allowed himself to get a clear and full idea of it, it might not be worth while. Stubbornness shows itself even more in

external; they are shifting things about. No ideal reward, no enrichment of emotion and intellect, accompanies them. Others contribute to the maintenance of life, and to its external adornment and display. Many of our existing social activities, industrial and political, fall in these two classes. Neither the people who engage in them, nor those who are directly affected by them, are capable of full and free interest in their work. Because of the lack of any purpose in the work for the one doing it, or because of the restricted character of its aim, intelligence is not adequately engaged. The same conditions force many people back upon themselves. They take refuge in an inner play of sentiment and fancies. They are aesthetic but not artistic, since their feelings and ideas are turned upon themselves, instead of being methods in acts which modify conditions. Their mental life is sentimental; an enjoyment of an inner landscape. Even the pursuit of science may become an asylum of refuge from the hard conditions of lifenot a temporary retreat for the sake of recuperation and clarification in future dealings with the world. The very word art may become associated not with specific transformation of things, making them more significant for mind, but with stimulations of eccentric fancy and with emotional indulgences. The separation and mutual contempt of the "practical" man and the man of theory or culture, the divorce of fine and industrial arts, are indications of this situation.

only get rid of the artificial man-imposed coercive restrictions.

Education in accord with nature was thought to be the first step in insuring this more social society. It was plainly seen that economic and political limitations were ultimately dependent upon limitations of thought and feeling. The first step in freeing men from external chains was to emancipate them from the internal chains of false beliefs and ideals. What was called social life, existing institutions, were too false and corrupt to be intrusted with this work. How could it be expected to undertake it when the undertaking meant its own destruction? "Nature" must then be the power to which the enterprise was to be left. Even the extreme sensationalistic theory of knowledge which was current derived itself from this conception. To insist that mind is originally passive and empty was one way of glorifying the possibilities of education. If the mind was a wax tablet to be written upon by objects, there were no limits to the possibility of education by means of the natural environment. And since the natural world of objects is a scene of harmonious "truth," this education would infallibly produce minds filled with the truth.
5. Education as National and as Social. As soon as the first enthusiasm for freedom waned, the weakness of the theory upon the constructive side became obvious. Merely to leave everything to

ground that life and instinct are a kind of miraculous thing anyway. Thus we fail to note what the essential characteristic of the event is; namely, the significance of the temporal place and order of each element; the way each prior event leads into its successor while the successor takes up what is furnished and utilizes it for some other stage, until we arrive at the end, which, as it were, summarizes and finishes off the process. Since aims relate always to results, the first thing to look to when it is a question of aims, is whether the work assigned possesses intrinsic continuity. Or is it a mere serial aggregate of acts, first doing one thing and then another? To talk about an educational aim when approximately each act of a pupil is dictated by the teacher, when the only order in the sequence of his acts is that which comes from the assignment of lessons and the giving of directions by another, is to talk nonsense. It is equally fatal to an aim to permit capricious or discontinuous action in the name of spontaneous self-expression. An aim implies an orderly and ordered activity, one in which the order consists in the progressive completing of a process. Given an activity having a time span and cumulative growth within the time succession, an aim means foresight in advance of the end or possible termination. If bees anticipated the consequences of their activity, if they perceived their end in imaginative foresight, they would have the primary element in an aim. Hence it is nonsense

thoroughgoing "disciplinary" subordination to existing institutions. The extent of the transformation of educational philosophy which occurred in Germany in the generation occupied by the struggle against Napoleon for national independence, may be gathered from Kant, who well expresses the earlier individualcosmopolitan ideal. In his treatise on Pedagogics, consisting of lectures given in the later years of the eighteenth century, he defines education as the process by which man becomes man. Mankind begins its history submerged in naturenot as Man who is a creature of reason, while nature furnishes only instinct and appetite. Nature offers simply the germs which education is to develop and perfect. The peculiarity of truly human life is that man has to create himself by his own voluntary efforts; he has to make himself a truly moral, rational, and free being. This creative effort is carried on by the educational activities of slow generations. Its acceleration depends upon men consciously striving to educate their successors not for the existing state of affairs but so as to make possible a future better humanity. But there is the great difficulty. Each generation is inclined to educate its young so as to get along in the present world instead of with a view to the proper end of education: the promotion of the best possible realization of humanity as humanity. Parents educate their children so that they may get on; princes educate

reluctance to criticize ends which present themselves than it does in persistence and energy in use of means to achieve the end. The really executive man is a man who ponders his ends, who makes his ideas of the results of his actions as clear and full as possible. The people we called weak-willed or self-indulgent always deceive themselves as to the consequences of their acts. They pick out some feature which is agreeable and neglect all attendant circumstances. When they begin to act, the disagreeable results they ignored begin to show themselves. They are discouraged, or complain of being thwarted in their good purpose by a hard fate, and shift to some other line of action. That the primary difference between strong and feeble volition is intellectual, consisting in the degree of persistent firmness and fullness with which consequences are thought out, cannot be over-emphasized.
(ii) There is, of course, such a thing as a speculative tracing out of results. Ends are then foreseen, but they do not lay deep hold of a person. They are something to look at and for curiosity to play with rather than something to achieve. There is no such thing as overintellectuality, but there is such a thing as a onesided intellectuality. A person "takes it out" as we say in considering the consequences of proposed lines of action. A certain flabbiness of fiber prevents the contemplated object from gripping him and engaging him in action. And most

situation of human intercourse. On the one hand, science, commerce, and art transcend national boundaries. They are largely international in quality and method. They involve interdependencies and cooperation among the peoples inhabiting different countries. At the same time, the idea of national sovereignty has never been as accentuated in politics as it is at the present time. Each nation lives in a state of suppressed hostility and incipient war with its neighbors. Each is supposed to be the supreme judge of its own interests, and it is assumed as matter of course that each has interests which are exclusively its own. To question this is to question the very idea of national sovereignty which is assumed to be basic to political practice and political science. This contradiction (for it is nothing less) between the wider sphere of associated and mutually helpful social life and the narrower sphere of exclusive and hence potentially hostile pursuits and purposes, exacts of educational theory a clearer conception of the meaning of "social" as a function and test of education than has yet been attained. Is it possible for an educational system to be conducted by a national state and yet the full social ends of the educative process not be restricted, constrained, and corrupted? Internally, the question has to face the tendencies, due to present economic conditions, which split society into classes some of which are made merely tools for the higher

indifferently and miscellaneously to any and every detail. It is centered upon whatever has a bearing upon the effective pursuit of your occupation. Your look is ahead, and you are concerned to note the existing facts because and in so far as they are factors in the achievement of the result intended. You have to find out what your resources are, what conditions are at command, and what the difficulties and obstacles are. This foresight and this survey with reference to what is foreseen constitute mind. Action that does not involve such a forecast of results and such an examination of means and hindrances is either a matter of habit or else it is blind. In neither case is it intelligent. To be vague and uncertain as to what is intended and careless in observation of conditions of its realization is to be, in that degree, stupid or partially intelligent.

If we recur to the case where mind is not concerned with the physical manipulation of the instruments but with what one intends to write, the case is the same. There is an activity in process; one is taken up with the development of a theme. Unless one writes as a phonograph talks, this means intelligence; namely, alertness in foreseeing the various conclusions to which present data and considerations are tending, together with continually renewed observation and recollection to get hold of the subject matter which bears upon the conclusions to be reached.

The account of education given in our earlier chapters virtually anticipated the results reached in a discussion of the purport of education in a democratic community. For it assumed that the aim of education is to enable individuals to continue their education-or that the object and reward of learning is continued capacity for growth. Now this idea cannot be applied to all the members of a society except where intercourse of man with man is mutual, and except where there is adequate provision for the reconstruction of social habits and institutions by means of wide stimulation arising from equitably distributed interests. And this means a democratic society. In our search for aims in education, we are not concerned, therefore, with finding an end outside of the educative process to which education is subordinate. Our whole conception forbids. We are rather concerned with the contrast which exists when aims belong within the process in which they operate and when they are set up from without. And the latter state of affairs must obtain when social relationships are not equitably balanced. For in that case, some portions of the whole social group will find their aims determined by an external dictation; their aims will not arise from the free growth of their own experience, and their nominal aims will be means to more ulterior ends of others rather than truly their own.

combination of the two. Subject matter is then regarded as something complete in itself; it is just something to be learned or known, either by the voluntary application of mind to it or through the impressions it makes on mind.

The facts of interest show that these conceptions are mythical. Mind appears in experience as ability to respond to present stimuli on the basis of anticipation of future possible consequences, and with a view to controlling the kind of consequences that are to take place. The things, the subject matter known, consist of whatever is recognized as having a bearing upon the anticipated course of events, whether assisting or retarding it. These statements are too formal to be very intelligible. An illustration may clear up their significance. You are engaged in a certain occupation, say writing with a typewriter. If you are an expert, your formed habits take care of the physical movements and leave your thoughts free to consider your topic. Suppose, however, you are not skilled, or that, even if you are, the machine does not work well. You then have to use intelligence. You do not wish to strike the keys at random and let the consequences be what they may; you wish to record certain words in a given order so as to make sense. You attend to the keys, to what you have written, to your movements, to the ribbon or the mechanism of the machine. Your attention is not distributed

because after the act is performed we note results which we had not noted before. But much work in school consists in setting up rules by which pupils are to act of such a sort that even after pupils have acted, they are not led to see the connection between the result-say the answer-and the method pursued. So far as they are concerned, the whole thing is a trick and a kind of miracle. Such action is essentially capricious, and leads to capricious habits. (b) Routine action, action which is automatic, may increase skill to do a particular thing. In so far, it might be said to have an educative effect. But it does not lead to new perceptions of bearings and connections; it limits rather than widens the meaning-horizon. And since the environment changes and our way of acting has to be modified in order successfully to keep a balanced connection with things, an isolated uniform way of acting becomes disastrous at some critical moment. The vaunted "skill" turns out gross ineptitude.

The essential contrast of the idea of education as continuous reconstruction with the other onesided conceptions which have been criticized in this and the previous chapter is that it identifies the end (the result) and the process. This is verbally self-contradictory, but only verbally. It means that experience as an active process occupies time and that its later period completes its earlier portion; it brings to light connections

But the idea which underlies it is that education is essentially retrospective; that it looks primarily to the past and especially to the literary products of the past, and that mind is adequately formed in the degree in which it is patterned upon the spiritual heritage of the past. This idea has had such immense influence upon higher instruction especially, that it is worth examination in its extreme formulation.

In the first place, its biological basis is fallacious. Embyronic growth of the human infant preserves, without doubt, some of the traits of lower forms of life. But in no respect is it a strict traversing of past stages. If there were any strict "law" of repetition, evolutionary development would clearly not have taken place. Each new generation would simply have repeated its predecessors' existence. Development, in short, has taken place by the entrance of shortcuts and alterations in the prior scheme of growth. And this suggests that the aim of education is to facilitate such short-circuited growth. The great advantage of immaturity, educationally speaking, is that it enables us to emancipate the young from the need of dwelling in an outgrown past. The business of education is rather to liberate the young from reviving and retraversing the past than to lead them to a recapitulation of it. The social environment of the young is constituted by the presence and action of the habits of thinking

there is a disposition to take considerations which are dear to the hearts of adults and set them up as ends irrespective of the capacities of those educated. There is also an inclination to propound aims which are so uniform as to neglect the specific powers and requirements of an individual, forgetting that all learning is something which happens to an individual at a given time and place. The larger range of perception of the adult is of great value in observing the abilities and weaknesses of the young, in deciding what they may amount to. Thus the artistic capacities of the adult exhibit what certain tendencies of the child are capable of; if we did not have the adult achievements we should be without assurance as to the significance of the drawing, reproducing, modeling, coloring activities of childhood. So if it were not for adult language, we should not be able to see the import of the babbling impulses of infancy. But it is one thing to use adult accomplishments as a context in which to place and survey the doings of childhood and youth; it is quite another to set them up as a fixed aim without regard to the concrete activities of those educated.
(2) An aim must be capable of translation into a method of cooperating with the activities of those undergoing instruction. It must suggest the kind of environment needed to liberate and to organize their capacities. Unless it lends itself to the

And it is well to remind ourselves that education as such has no aims. Only persons, parents, and teachers, etc., have aims, not an abstract idea like education. And consequently their purposes are indefinitely varied, differing with different children, changing as children grow and with the growth of experience on the part of the one who teaches. Even the most valid aims which can be put in words will, as words, do more harm than good unless one recognizes that they are not aims, but rather suggestions to educators as to how to observe, how to look ahead, and how to choose in liberating and directing the energies of the concrete situations in which they find themselves. As a recent writer has said: "To lead this boy to read Scott's novels instead of old Sleuth's stories; to teach this girl to sew; to root out the habit of bullying from John's make-up; to prepare this class to study medicine,-these are samples of the millions of aims we have actually before us in the concrete work of education." Bearing these qualifications in mind, we shall proceed to state some of the characteristics found in all good educational aims. (1) An educational aim must be founded upon the intrinsic activities and needs (including original instincts and acquired habits) of the given individual to be educated. The tendency of such an aim as preparation is, as we have seen, to omit existing powers, and find the aim in some remote accomplishment or responsibility. In general,

adequate interplay of experiences-the more action tends to become routine on the part of the class at a disadvantage, and capricious, aimless, and explosive on the part of the class having the materially fortunate position. Plato defined a slave as one who accepts from another the purposes which control his conduct. This condition obtains even where there is no slavery in the legal sense. It is found wherever men are engaged in activity which is socially serviceable, but whose service they do not understand and have no personal interest in. Much is said about scientific management of work. It is a narrow view which restricts the science which secures efficiency of operation to movements of the muscles. The chief opportunity for science is the discovery of the relations of a man to his workincluding his relations to others who take partwhich will enlist his intelligent interest in what he is doing. Efficiency in production often demands division of labor. But it is reduced to a mechanical routine unless workers see the technical, intellectual, and social relationships involved in what they do, and engage in their work because of the motivation furnished by such perceptions. The tendency to reduce such things as efficiency of activity and scientific management to purely technical externals is evidence of the one-sided stimulation of thought given to those in control of industry-those who supply its aims. Because of their lack of all-round and well-balanced social

perfunctory and superficial where there is no interest. Parents and teachers often complainand correctly-that children "do not want to hear, or want to understand." Their minds are not upon the subject precisely because it does not touch them; it does not enter into their concerns. This is a state of things that needs to be remedied, but the remedy is not in the use of methods which increase indifference and aversion. Even punishing a child for inattention is one way of trying to make him realize that the matter is not a thing of complete unconcern; it is one way of arousing "interest," or bringing about a sense of connection. In the long run, its value is measured by whether it supplies a mere physical excitation to act in the way desired by the adult or whether it leads the child "to think"-that is, to reflect upon his acts and impregnate them with aims.
(ii) That interest is requisite for executive persistence is even more obvious. Employers do not advertise for workmen who are not interested in what they are doing. If one were engaging a lawyer or a doctor, it would never occur to one to reason that the person engaged would stick to his work more conscientiously if it was so uncongenial to him that he did it merely from a sense of obligation. Interest measures-or rather is-the depth of the grip which the foreseen end has upon one, moving one to act for its realization.
2. The Importance of the Idea of Interest in

to it laden with the spoils of the past. A mind that is adequately sensitive to the needs and occasions of the present actuality will have the liveliest of motives for interest in the background of the present, and will never have to hunt for a way back because it will never have lost connection.
3. Education as Reconstruction. In its contrast with the ideas both of unfolding of latent powers from within, and of the formation from without, whether by physical nature or by the cultural products of the past, the ideal of growth results in the conception that education is a constant reorganizing or reconstructing of experience. It has all the time an immediate end, and so far as activity is educative, it reaches that end-the direct transformation of the quality of experience. Infancy, youth, adult life,-all stand on the same educative level in the sense that what is really learned at any and every stage of experience constitutes the value of that experience, and in the sense that it is the chief business of life at every point to make living thus contribute to an enrichment of its own perceptible meaning.

We thus reach a technical definition of education: It is that reconstruction or reorganization of experience which adds to the meaning of experience, and which increases ability to direct the course of subsequent experience. (1) The increment of meaning corresponds to the increased perception of the

Much which has been said so far is borrowed from what Plato first consciously taught the world. But conditions which he could not intellectually control led him to restrict these ideas in their application. He never got any conception of the indefinite plurality of activities which may characterize an individual and a social group, and consequently limited his view to a limited number of classes of capacities and of social arrangements. Plato's starting point is that the organization of society depends ultimately upon knowledge of the end of existence. If we do not know its end, we shall be at the mercy of accident and caprice. Unless we know the end, the good, we shall have no criterion for rationally deciding what the possibilities are which should be promoted, nor how social arrangements are to be ordered. We shall have no conception of the proper limits and distribution of activities-what he called justice-as a trait of both individual and social organization. But how is the knowledge of the final and permanent good to be achieved? In dealing with this question we come upon the seemingly insuperable obstacle that such knowledge is not possible save in a just and harmonious social order. Everywhere else the mind is distracted and misled by false valuations and false perspectives. A disorganized and factional society sets up a number of different models and standards. Under such conditions it is impossible for the individual to attain consistency

importance of what has been taught consists in its availability for further teaching, reflects the pedagogue's view of life. The philosophy is eloquent about the duty of the teacher in instructing pupils; it is almost silent regarding his privilege of learning. It emphasizes the influence of intellectual environment upon the mind; it slurs over the fact that the environment involves a personal sharing in common experiences. It exaggerates beyond reason the possibilities of consciously formulated and used methods, and underestimates the role of vital, unconscious, attitudes. It insists upon the old, the past, and passes lightly over the operation of the genuinely novel and unforeseeable. It takes, in brief, everything educational into account save its essence,-vital energy seeking opportunity for effective exercise. All education forms character, mental and moral, but formation consists in the selection and coordination of native activities so that they may utilize the subject matter of the social environment. Moreover, the formation is not only a formation of native activities, but it takes place through them. It is a process of reconstruction, reorganization.
2. Education as Recapitulation and Retrospection. A peculiar combination of the ideas of development and formation from without has given rise to the recapitulation theory of education, biological and cultural. The individual

differences of endowment the dynamic values of natural inequalities of growth, and utilize them, preferring irregularity to the rounding out gained by pruning will most closely follow that which takes place in the body and thus prove most effective." 1 Observation of natural tendencies is difficult under conditions of restraint. They show themselves most readily in a child's spontaneous sayings and doings,-that is, in those he engages in when not put at set tasks and when not aware of being under observation. It does not follow that these tendencies are all desirable because they are natural; but it does follow that since they are there, they are operative and must be taken account of. We must see to it that the desirable ones have an environment which keeps them active, and that their activity shall control the direction the others take and thereby induce the disuse of the latter because they lead to nothing. Many tendencies that trouble parents when they appear are likely to be transitory, and sometimes too much direct attention to them only fixes a child's attention upon them. At all events, adults too easily assume their own habits and wishes as standards, and regard all deviations of children's impulses as evils to be eliminated. That artificiality against which the conception of following nature is so largely a protest, is the outcome of attempts to force children directly into the mold of grown-up standards.

efficiency is identified with a narrow range of acts, instead of with the spirit and meaning of activity, culture is opposed to efficiency. Whether called culture or complete development of personality, the outcome is identical with the true meaning of social efficiency whenever attention is given to what is unique in an individual-and he would not be an individual if there were not something incommensurable about him. Its opposite is the mediocre, the average. Whenever distinctive quality is developed, distinction of personality results, and with it greater promise for a social service which goes beyond the supply in quantity of material commodities. For how can there be a society really worth serving unless it is constituted of individuals of significant personal qualities?

The fact is that the opposition of high worth of personality to social efficiency is a product of a feudally organized society with its rigid division of inferior and superior. The latter are supposed to have time and opportunity to develop themselves as human beings; the former are confined to providing external products. When social efficiency as measured by product or output is urged as an ideal in a would-be democratic society, it means that the depreciatory estimate of the masses characteristic of an aristocratic community is accepted and carried over. But if democracy has a moral and ideal meaning, it is that a social return be demanded from all and

lived in a dumb unsocial environment where men refused to talk to one another and used only that minimum of gestures without which they could not get along, vocal language would be as unachieved by him as if he had no vocal organs. If the sounds which he makes occur in a medium of persons speaking the Chinese language, the activities which make like sounds will be selected and coordinated. This illustration may be applied to the entire range of the educability of any individual. It places the heritage from the past in its right connection with the demands and opportunities of the present.
(2) The theory that the proper subject matter of instruction is found in the culture-products of past ages (either in general, or more specifically in the particular literatures which were produced in the culture epoch which is supposed to correspond with the stage of development of those taught) affords another instance of that divorce between the process and product of growth which has been criticized. To keep the process alive, to keep it alive in ways which make it easier to keep it alive in the future, is the function of educational subject matter. But an individual can live only in the present. The present is not just something which comes after the past; much less something produced by it. It is what life is in leaving the past behind it. The study of past products will not help us understand the present, because the present is

intelligence because, given ready-made, they must be imposed by some authority external to intelligence, leaving to the latter nothing but a mechanical choice of means.
(2) We have spoken as if aims could be completely formed prior to the attempt to realize them. This impression must now be qualified. The aim as it first emerges is a mere tentative sketch. The act of striving to realize it tests its worth. If it suffices to direct activity successfully, nothing more is required, since its whole function is to set a mark in advance; and at times a mere hint may suffice. But usually-at least in complicated situations-acting upon it brings to light conditions which had been overlooked. This calls for revision of the original aim; it has to be added to and subtracted from. An aim must, then, be flexible; it must be capable of alteration to meet circumstances. An end established externally to the process of action is always rigid. Being inserted or imposed from without, it is not supposed to have a working relationship to the concrete conditions of the situation. What happens in the course of action neither confirms, refutes, nor alters it. Such an end can only be insisted upon. The failure that results from its lack of adaptation is attributed simply to the perverseness of conditions, not to the fact that the end is not reasonable under the circumstances. The value of a legitimate aim, on the contrary, lies

involved, but hitherto unperceived. The later outcome thus reveals the meaning of the earlier, while the experience as a whole establishes a bent or disposition toward the things possessing this meaning. Every such continuous experience or activity is educative, and all education resides in having such experiences.

It remains only to point out (what will receive more ample attention later) that the reconstruction of experience may be social as well as personal. For purposes of simplification we have spoken in the earlier chapters somewhat as if the education of the immature which fills them with the spirit of the social group to which they belong, were a sort of catching up of the child with the aptitudes and resources of the adult group. In static societies, societies which make the maintenance of established custom their measure of value, this conception applies in the main. But not in progressive communities. They endeavor to shape the experiences of the young so that instead of reproducing current habits, better habits shall be formed, and thus the future adult society be an improvement on their own. Men have long had some intimation of the extent to which education may be consciously used to eliminate obvious social evils through starting the young on paths which shall not produce these ills, and some idea of the extent in which education may be made an instrument of realizing the

- And Image files...

In [ ]:
image_file = Path("receipt.png")
assert image_file.is_file()

In [ ]:
import base64

encoded = base64.b64encode(image_file.read_bytes()).decode()
base64_data_url = f"data:image/jpeg;base64,{encoded}"

image_response = client.ocr.process(document=ImageURLChunk(image_url=base64_data_url), model="mistral-ocr-latest")

response_dict = json.loads(image_response.json())
json_string = json.dumps(response_dict, indent=4)
print(json_string)

We want to be able to extract structured data from these files. For this, we will make use of `pixtral-12b-latest` and support it with our OCR model for better, high-quality answers.

In [ ]:
image_ocr_markdown = image_response.pages[0].markdown

chat_response = client.chat.complete(
    model="pixtral-12b-latest",
    messages=[
        {
            "role": "user",
            "content": [
                ImageURLChunk(image_url=base64_data_url),
                TextChunk(text=f"This is image's OCR in markdown:\n<BEGIN_IMAGE_OCR>\n{image_ocr_markdown}\n<END_IMAGE_OCR>.\nConvert this into a sensible structured json response. The output should be strictly be json with no extra commentary")
            ],
        },
    ],
    response_format =  {"type": "json_object"},
    temperature=0
)

response_dict = json.loads(chat_response.choices[0].message.content)
json_string = json.dumps(response_dict, indent=4)
print(json_string)

Note: We are leveraging a model already capable of vision tasks. However, we could also use text-only models for the structured output.

In [ ]:
image_ocr_markdown = image_response.pages[0].markdown

chat_response = client.chat.complete(
    model="ministral-8b-latest",
    messages=[
        {
            "role": "user",
            "content": f"This is image's OCR in markdown:\n<BEGIN_IMAGE_OCR>\n{image_ocr_markdown}\n<END_IMAGE_OCR>.\nConvert this into a sensible structured json response. The output should be strictly be json with no extra commentary"
        },
    ],
    response_format =  {"type": "json_object"},
    temperature=0
)

response_dict = json.loads(chat_response.choices[0].message.content)
json_string = json.dumps(response_dict, indent=4)
print(json_string)

### All Together
Let's design a simple function that takes an `image_path` file and returns a JSON structured output in a specific format. In this case, we arbitrarily decided we wanted an output respecting the following:

```python
class StructuredOCR:
    file_name: str  # can be any string
    topics: list[str]  # must be a list of strings
    languages: list[Language]  # a list of languages
    ocr_contents: dict  # any dictionary, can be freely defined by the model
```

We will make use of [custom structured outputs](https://docs.mistral.ai/capabilities/structured-output/custom_structured_output/) as well as `pycountry` for the languages.

In [ ]:
!pip install pycountry

In [ ]:
from enum import Enum
from pathlib import Path
from pydantic import BaseModel
import base64
import pycountry

languages = {lang.alpha_2: lang.name for lang in pycountry.languages if hasattr(lang, 'alpha_2')}

class LanguageMeta(Enum.__class__):
    def __new__(metacls, cls, bases, classdict):
        for code, name in languages.items():
            classdict[name.upper().replace(' ', '_')] = name
        return super().__new__(metacls, cls, bases, classdict)

class Language(Enum, metaclass=LanguageMeta):
    pass

class StructuredOCR(BaseModel):
    file_name: str
    topics: list[str]
    languages: list[Language]
    ocr_contents: dict

print(StructuredOCR.schema_json())

def structured_ocr(image_path: str) -> StructuredOCR:
    image_file = Path(image_path)
    assert image_file.is_file(), "The provided image path does not exist."

    # Read and encode the image file
    encoded_image = base64.b64encode(image_file.read_bytes()).decode()
    base64_data_url = f"data:image/jpeg;base64,{encoded_image}"

    # Process the image using OCR
    image_response = client.ocr.process(document=ImageURLChunk(image_url=base64_data_url), model="mistral-ocr-latest")
    image_ocr_markdown = image_response.pages[0].markdown

    # Parse the OCR result into a structured JSON response
    chat_response = client.chat.parse(
        model="pixtral-12b-latest",
        messages=[
            {
                "role": "user",
                "content": [
                    ImageURLChunk(image_url=base64_data_url),
                    TextChunk(text=(
                        "This is the image's OCR in markdown:\n"
                        f"<BEGIN_IMAGE_OCR>\n{image_ocr_markdown}\n<END_IMAGE_OCR>.\n"
                        "Convert this into a structured JSON response with the OCR contents in a sensible dictionnary."
                    ))
                ],
            },
        ],
        response_format=StructuredOCR,
        temperature=0
    )

    return chat_response.choices[0].message.parsed

We can now extract structured output from any image parsed with our OCR model.

In [ ]:
image_path = "receipt.png"
structured_response = structured_ocr(image_path)

response_dict = json.loads(structured_response.json())
json_string = json.dumps(response_dict, indent=4)
print(json_string)

The original image for comparison can be found below.

In [ ]:
from PIL import Image

image = Image.open(image_path)
image